# Halo Model: CIB, tSZ, and CIB x tSZ Power Spectra

This notebook demonstrates how to compute CIB, tSZ, and CIB x tSZ angular power spectra using the `halomodel_cib_tsz` package.

Based on [Maniyar, Bethermin & Lagache (2021)](https://arxiv.org/abs/2006.16329).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.dpi'] = 120

## 1. Compute all spectra with one call

The simplest way to use the package is via `compute_spectra()`. By default it computes CIB, tSZ, and CIB x tSZ for Planck frequencies (100, 143, 217, 353, 545, 857 GHz).

In [ ]:
from halomodel_cib_tsz import compute_spectra

result = compute_spectra(
    components=('cib', 'tsz', 'cibxtsz'),
    experiment='Planck',
)

ell = result['ell']
freqs = result['freqs']
print(f"Frequencies: {freqs} GHz")
print(f"Multipoles: {ell[0]:.0f} to {ell[-1]:.0f} ({len(ell)} points)")

## 2. CIB power spectra

The CIB power spectra are stored as `(n_freq, n_freq, n_ell)` arrays. For Planck, the frequency indices are:

| Index | Frequency |
|-------|-----------|
| 0 | 100 GHz |
| 1 | 143 GHz |
| 2 | 217 GHz |
| 3 | 353 GHz |
| 4 | 545 GHz |
| 5 | 857 GHz |

In [ ]:
freq_labels = ['100', '143', '217', '353', '545', '857']

fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

# Plot three auto-spectra
for ax, idx, color in zip(axes, [2, 4, 5], ['C0', 'C1', 'C2']):
    ax.loglog(ell, result['cl_cib_1h'][idx, idx, :], '--', color=color, label='1-halo')
    ax.loglog(ell, result['cl_cib_2h'][idx, idx, :], ':', color=color, label='2-halo')
    ax.loglog(ell, result['cl_cib'][idx, idx, :], '-', color=color, label='total')
    ax.set_xlabel(r'$\ell$')
    ax.set_ylabel(r'$C_\ell$ [Jy$^2$/sr]')
    ax.set_title(f'CIB {freq_labels[idx]} x {freq_labels[idx]} GHz')
    ax.legend(frameon=False, fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Mean CIB intensity

In [ ]:
print("Mean CIB intensity <I_nu> [nW/m^2/sr]:")
print("-" * 35)
for i, f in enumerate(freq_labels):
    print(f"  {f:>4s} GHz:  {result['I_nu'][i]:.4f}")

## 3. tSZ power spectrum

The tSZ spectrum is plotted as the dimensionless $D_\ell^{yy} = \ell(\ell+1) C_\ell^{yy} / 2\pi$. We extract the yy spectrum by dividing out the spectral function factors.

In [ ]:
from halomodel_cib_tsz import config

# Extract C_l^yy from the 100x100 GHz channel
# The stored spectra have units Jy^2/sr and include f_nu * Kcmb_MJy factors
fnu = config.PLANCK['f_nu_bandpassed']
Kcmb_MJy = config.PLANCK['Kcmb_MJy']
conv = (fnu[0] * 1e6 * Kcmb_MJy[0])**2  # conversion factor for 100 GHz

cl_yy_1h = result['cl_tsz_1h'][0, 0, :] / conv
cl_yy_2h = result['cl_tsz_2h'][0, 0, :] / conv
cl_yy = cl_yy_1h + cl_yy_2h

# D_ell = ell*(ell+1)*C_ell / (2*pi)
dl_factor = ell * (ell + 1) / (2 * np.pi)

fig, ax = plt.subplots(figsize=(7, 5))
ax.loglog(ell, dl_factor * cl_yy_1h, 'r--', label='1-halo')
ax.loglog(ell, dl_factor * cl_yy_2h, 'r:', label='2-halo')
ax.loglog(ell, dl_factor * cl_yy, 'r-', lw=2, label='total')
ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$D_\ell^{yy} = \ell(\ell+1) C_\ell^{yy} / 2\pi$')
ax.set_title('tSZ power spectrum')
ax.legend(frameon=False)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. CIB x tSZ cross-power spectrum

The cross-spectrum can be positive or negative depending on the frequency pair. At 143 GHz it is negative (tSZ decrement), at 857 GHz it is positive.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4.5))

pairs = [(1, 5, '143 x 857'), (5, 5, '857 x 857')]
colors = ['C3', 'C4']

for ax, (i, j, label), color in zip(axes, pairs, colors):
    cl_1h = result['cl_cibxtsz_1h'][i, j, :]
    cl_2h = result['cl_cibxtsz_2h'][i, j, :]
    cl_tot = result['cl_cibxtsz'][i, j, :]

    ax.loglog(ell, np.abs(cl_1h), '--', color=color, label='|1-halo|')
    ax.loglog(ell, np.abs(cl_2h), ':', color=color, label='|2-halo|')
    ax.loglog(ell, np.abs(cl_tot), '-', color=color, lw=2, label='|total|')

    sign = 'positive' if cl_tot[0] > 0 else 'negative'
    ax.set_xlabel(r'$\ell$')
    ax.set_ylabel(r'$|C_\ell|$ [Jy$^2$/sr]')
    ax.set_title(f'CIB x tSZ {label} GHz ({sign})')
    ax.legend(frameon=False, fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Computing only specific components

If you only need CIB spectra, skip tSZ and cross to save time:

In [ ]:
%%time
result_cib = compute_spectra(components=('cib',), experiment='Planck')

print(f"Keys returned: {list(result_cib.keys())}")
print(f"857x857 CIB at ell={result_cib['ell'][5]:.0f}: {result_cib['cl_cib'][5, 5, 5]:.1f} Jy^2/sr")

## 6. Custom CIB parameters

You can pass your own CIB model parameters. The four parameters control the star formation efficiency as a function of halo mass:

- `Meff`: halo mass at peak efficiency [M_sun]
- `eta_max`: maximum SFR efficiency
- `sigma_Mh`: log-normal width
- `tau`: controls late-time (low-z) evolution

In [ ]:
# Default parameters
default_params = {'Meff': 5.34e12, 'eta_max': 0.513, 'sigma_Mh': 1.248, 'tau': 0.826}

# Higher peak mass
alt_params = {'Meff': 1e13, 'eta_max': 0.513, 'sigma_Mh': 1.248, 'tau': 0.826}

r_default = compute_spectra(components=('cib',), cib_params=default_params)
r_alt = compute_spectra(components=('cib',), cib_params=alt_params)

fig, ax = plt.subplots(figsize=(7, 5))
idx = 5  # 857 GHz
ax.loglog(r_default['ell'], r_default['cl_cib'][idx, idx, :], 'b-',
          label=f'M_eff = {default_params["Meff"]:.2e} M_sun')
ax.loglog(r_alt['ell'], r_alt['cl_cib'][idx, idx, :], 'r--',
          label=f'M_eff = {alt_params["Meff"]:.2e} M_sun')
ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$C_\ell$ [Jy$^2$/sr]')
ax.set_title('CIB 857 x 857 GHz: effect of M_eff')
ax.legend(frameon=False)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Increasing grid resolution

For publication-quality results, increase the grid sizes. This makes the computation slower but more accurate.

In [ ]:
%%time
result_hires = compute_spectra(
    components=('cib',),
    n_mass=150,
    n_z=150,
    n_ell=200,
)

fig, ax = plt.subplots(figsize=(7, 5))
idx = 5  # 857 GHz
ax.loglog(result['ell'], result['cl_cib'][idx, idx, :], 'b-', alpha=0.5,
          label=f'default (n_ell={len(result["ell"])})')
ax.loglog(result_hires['ell'], result_hires['cl_cib'][idx, idx, :], 'r-',
          label=f'high-res (n_ell={len(result_hires["ell"])})')
ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$C_\ell$ [Jy$^2$/sr]')
ax.set_title('CIB 857 x 857 GHz: grid convergence')
ax.legend(frameon=False)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()